In [3]:
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import os
import cv2
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from tensorflow.keras.optimizers import Adam

In [134]:
# Define the base directory path containing the dataset of images.
# The dataset is organized into two categories: 'Male' and 'Female'.

path1 = r"C:\Male and Female"
cate = ['Female', 'Male']

In [136]:
image_size = 200 # Set the target image size for resizing all images in the dataset.
input_image = [] # Initialize an empty list to store the image data along with their corresponding labels.

# Loop through each category in the dataset (e.g., 'Mask' and 'No_Mask').
for i in cate:
    folders = os.path.join(path1, i) # Construct the full path to the folder corresponding to the current category.
    label = cate.index(i)   # Assign a numeric label to the category (e.g., 0 for 'Mask', 1 for 'No_Mask').

    # Iterate through all the images in the category's folder.
    for image in os.listdir(folders):
        image_path = os.path.join(folders, image)
        image_array = cv2.imread(image_path)
        image_array = cv2.resize(image_array, (image_size, image_size))
        input_image.append([image_array, label])

In [138]:
input_image[1000]

[array([[[ 95, 121, 133],
         [ 43,  72,  88],
         [ 40,  47,  76],
         ...,
         [ 26,  34,  64],
         [ 24,  30,  61],
         [ 39,  47,  78]],
 
        [[ 80, 118, 130],
         [ 40,  65,  87],
         [ 39,  44,  75],
         ...,
         [ 24,  33,  70],
         [ 33,  46,  82],
         [ 45,  58,  90]],
 
        [[ 64, 101, 113],
         [ 35,  51,  75],
         [ 35,  47,  75],
         ...,
         [ 27,  35,  73],
         [ 25,  39,  71],
         [ 38,  60,  88]],
 
        ...,
 
        [[137, 154, 207],
         [133, 152, 204],
         [130, 151, 205],
         ...,
         [155, 185, 236],
         [149, 182, 232],
         [151, 184, 234]],
 
        [[136, 158, 210],
         [129, 150, 203],
         [132, 153, 205],
         ...,
         [154, 185, 234],
         [152, 185, 233],
         [153, 186, 235]],
 
        [[134, 156, 208],
         [136, 157, 208],
         [134, 152, 205],
         ...,
         [155, 185, 234],
  

In [140]:
np.random.shuffle(input_image) # Shuffle the input_image list randomly to ensure the data is not ordered by category.

In [142]:
X = []  # Initialize empty lists to store features (X) and labels (Y)
Y = []  # Loop through each pair (X_values, labels) in the input_image dataset

for X_values, labels in input_image:
    X.append(X_values)
    Y.append(labels)

In [144]:
# Convert the list of features (X) into a NumPy array for efficient numerical operations
X = np.array(X)

# Convert the list of labels (Y) into a NumPy array for easier handling in machine learning algorithms
Y = np.array(Y)

In [146]:
# Splits the input_image dataset into training and testing subsets

train = input_image[0:4334]
test = input_image[4335:5418]

In [147]:
len(test)

1083

In [150]:
len(train), len(test)

(4334, 1083)

In [152]:
# Initialize empty lists to store training features (X_train) and training labels (Y_train)

X_train = []
Y_train = []

# Loop through each pair (X_values, labels) in the train dataset
for X_values, labels in train:
    
    X_train.append(X_values) # Append the feature values (X_values) to the X_train list
    
    Y_train.append(labels)  # Append the corresponding label to the Y_train list

In [154]:
# Initialize empty lists to store testing features (X_test) and testing labels (Y_test)
X_test = []
Y_test = []

# Loop through each pair (X_values, labels) in the test dataset
for X_values, labels in test:
    X_test.append(X_values)
    Y_test.append(labels)

In [156]:
# Convert the list of training features (X_train) into a NumPy array for efficient numerical operations
X_train = np.array(X_train)

# Convert the list of training labels (Y_train) into a NumPy array for easier handling in machine learning models
Y_train = np.array(Y_train)

# Convert the list of testing features (X_test) into a NumPy array for efficient numerical operations
X_test = np.array(X_test)

In [158]:
# Normalize the pixel values of the training set by dividing each pixel by 255 (scaling to the range [0, 1])
X_train = X_train / 255

# Normalize the pixel values of the testing set by dividing each pixel by 255 (scaling to the range [0, 1])
X_test = X_test / 255

In [160]:
# This code defines a Convolutional Neural Network (CNN) with four convolutional layers,
# each followed by max-pooling, and two fully connected layers for classifying images into two categories.

model = Sequential()
model.add(Conv2D(filters = 32, kernel_size =(5,5), activation = 'relu', padding = 'same',))
model.add(MaxPool2D(pool_size = (2,2)))

model.add(Conv2D(filters = 32, kernel_size =(5,5), activation = 'relu'))
model.add(MaxPool2D(pool_size = (2,2)))

model.add(Conv2D(filters = 32, kernel_size =(5,5), activation = 'relu'))
model.add(MaxPool2D(pool_size = (2,2)))

model.add(Conv2D(filters = 32, kernel_size =(5,5), activation = 'relu'))
model.add(MaxPool2D(pool_size = (2,2)))

model.add(Flatten())

model.add(Dense(128, activation = 'relu', input_shape = X_train.shape[1:]))
model.add(Dense(2, activation = 'softmax'))


C:\Users\ASUS\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [162]:
model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy']) # Compiling the model with the Adam optimizer, sparse categorical cross-entropy loss, and accuracy metric

In [164]:
model.fit(X_train, Y_train, batch_size=64, epochs = 10, validation_split= .1) # Training the model on the training data with the specified batch size, number of epochs, and validation split

Epoch 1/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 72s 1s/step - accuracy: 0.5981 - loss: 0.6566 - val_accuracy: 0.7512 - val_loss: 0.5090
Epoch 2/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 73s 919ms/step - accuracy: 0.7786 - loss: 0.4661 - val_accuracy: 0.8318 - val_loss: 0.3919
Epoch 3/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 56s 914ms/step - accuracy: 0.8669 - loss: 0.3156 - val_accuracy: 0.8756 - val_loss: 0.2810
Epoch 4/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 57s 940ms/step - accuracy: 0.9073 - loss: 0.2314 - val_accuracy: 0.9032 - val_loss: 0.2208
Epoch 5/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 59s 966ms/step - accuracy: 0.9460 - loss: 0.1357 - val_accuracy: 0.9539 - val_loss: 0.1569
Epoch 6/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 58s 952ms/step - accuracy: 0.9720 - loss: 0.0812 - val_accuracy: 0.9585 - val_loss: 0.1196
Epoch 7/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 60s 985ms/step - accuracy: 0.9906 - loss: 0.0319 - val_accuracy: 0.9700 - val_loss: 0.0964
Epoch 8/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 59s 961ms/step - accuracy: 0.9881 - loss: 0.0352 - val_accurac

In [166]:
# Use the trained model to predict the output (class probabilities) for the test set
pred_test = model.predict(X_test)

34/34 ━━━━━━━━━━━━━━━━━━━━ 4s 119ms/step


In [168]:
pred_test_cate = pred_test.argmax(axis = 1)  # Convert predicted probabilities to class labels by selecting the index of the highest probability

In [170]:
from sklearn.metrics import confusion_matrix, classification_report

In [172]:
tab = confusion_matrix(Y_test, pred_test_cate)  # Compute the confusion matrix to compare predicted vs. true labels
print(classification_report(Y_test, pred_test_cate))  # Print the classification report with precision, recall, F1-score, and support for each class

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       559
           1       0.98      0.98      0.98       524

    accuracy                           0.98      1083
   macro avg       0.98      0.98      0.98      1083
weighted avg       0.98      0.98      0.98      1083



In [178]:
tab.diagonal().sum()/tab.sum() # Calculate the accuracy by dividing the sum of correct predictions (diagonal elements) by the total number of predictions

0.9787626962142197

In [ ]:
model.save('Male_Female_model.h5') # Saving the model

In [4]:
# This code captures video from a webcam, detects faces using a Haar Cascade classifier,
# processes the faces through a pre-trained CNN model to predict if the person is male or female, 
# and displays the results with labeled rectangles around the faces.

from PIL import Image


model=load_model("Male_Female_model.h5")

results={0:'Female',1:'Male'}
GR_dict={0:(0,255,255),1:(0,255,0)}

rect_size = 4
cap = cv2.VideoCapture(0) 


haarcascade = cv2.CascadeClassifier(r"C:\Users\ASUS\OneDrive\Desktop\Imarticus AI and ML\Haarcascade\haarcascade_frontalface_default.xml")
while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1) 

    
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f]

        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(200,200)) #  same val as used in cnn
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,200,200,3)) #  same val as used in cnn
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)

        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,0,0),2)

    cv2.imshow('Liv Camera',   im)
    key = cv2.waitKey(10)
    
    if key == 27: # use the escape key
        break

cap.release()

cv2.destroyAllWindows()
        